# A signature scheme from the secant variety of the Grassmannian

In [22]:

def Plucker(v,w):
    
    
    """
    Plucker coordinates 
    """

    n = len(v)
    N =  binomial(n,2)
    F =  v[0].parent()
    y =  [F(0) for i in range(N)]
    m=0
    for i in range(n-1):
        for j in [i+1..n-1]:
            y[m] = v[i]*w[j]-v[j]*w[i]
            m= m+1
    return y 

In [23]:

def random_linear_form(R):
    
    """
    This function gives a random linear polynomial.
    """


    F = R.base_ring()
    ls = R.gens()
    f = 0
    for i in ls:
        f= f+F.random_element()*i
    return f

In [24]:

def random_sparse_matrix(F,n):
    
    """
    Function generating a nxn matrix with around 2n
    elements equal to 1 and the remaining are equal to 0
    """


    N = binomial(n,2)
    v = [F(0) for i in range(N)]
    m = int(N/n)
    for i in range(N):
        r = randint(0,m-1)
        if r == 0:
            v[i]=1
            
    # Upper triangular matrix            
    M = Matrix(F,n-1)  
    k = 0
    for i in range(n-1): 
        for j in [i..n-2]:
            M[i,j]=v[k]
            k=k+1      
    M = Matrix(F,n-1,1).augment(M)
    M = M.stack(Matrix(F,1,n)) + identity_matrix(F,n) 
    M.change_ring(F)
    S = Permutations([i+1 for i in range(n)])    
    A = Permutation(S.random_element()).to_matrix()
    A.change_ring(F)
    B = Permutation(S.random_element()).to_matrix()
    B.change_ring(F)
    M = A*M*B
    return M            
        

 # A toy example 

In [25]:
"""
Global Parameters
"""

F = GF(2)
K=GF(2^13)
n = 8 # dimension of a vector space V
d = 2*(n-2) # dimension of the Grassmannian as a variety : k*(n-k)
N = binomial(n,2) #dimension+1 of the Ambiant space
R = PolynomialRing(K,N,'x',order='degrevlex')
Ad = AffineSpace(K,n-2,'y')
Rd = Ad.coordinate_ring()
X = Matrix(N,1,R.gens())
m = n-2


In [26]:

"""

Private keys  = (MA,MAinv)
Public Keys  = KA 
"""

def KeygenA():
    
    m = n-2
    
    # Anti-symmetric matrix from the variables of R
    M = Matrix(R,n)
    ls = R.gens()
    count = 0
    for i in range(n):
        for j in [i+1..n-1]:
            count = count+1
            M[i,j] = ls[count-1]
    P = Matrix(R,n)
    for i in range(n):
        for j in range(n):
            if i<j:
                P[i,j] = M[i,j]
            if i>j:
                P[i,j] = -M.transpose()[i,j]            
    
    S = set([i for i in range(n)])
    S1 = Subsets(S,6)
    
    
    # equations defining Sec(G(2,n))
    eqs_Sec_of_Grass = []
    sub=[]
    for i in S1:
        sub.append(P[list(i),list(i)]) # submatrices from rows and column from list(x)
        
    for i in sub:
        eqs_Sec_of_Grass.append(i.pfaffian())
    

    # Alice's Private Keys MA, MAinv
    MA = random_sparse_matrix(F,N)
    MAinv = MA^(-1)
    
    MA.change_ring(R) 
    MAinv.change_ring(R) 
    
   
 
    v = vector(MA*X) # shifting
    m_set_of_equations = Subsets(eqs_Sec_of_Grass,m).random_element() # random m = n-2 equations from a set of defining equations of the Secant variety.
    KA = [] # Public Keys
    for f in m_set_of_equations:
        KA.append(f(*v))
    return KA , (MA,MAinv)
    

In [27]:
% time KA, (MA,MAinv)=KeygenA()

CPU times: user 1 s, sys: 48.4 ms, total: 1.05 s
Wall time: 1.32 s


In [28]:

def Signing(): 
    
    # Signing 

   
    D = [random_linear_form(R) for i in [1..n-2]]   # the document  (n-d) = n-2 linear equations
    X = Matrix(N,1,R.gens())
    v = vector(MAinv*X)
    DA =[]
    for f in D:
        DA.append(f.substitute(v))
    a = [Rd(0) for i in range(n)]
    a[0]=1
    SA = [[K(0) for i in range(N)] for j in range(2) ]
    for i in [2..n-1]:
        a[i] = Rd.gens()[i-2]
    for j in range(2):
        b = [K.random_element() for i in range(n)]
        b[0] = 0
        b[1] = 1
        y=vector(Plucker(a,b))   
        YA = []
        for f in DA:
            YA.append(f.substitute(y))
        YA = Ad.subscheme(YA)      
        YA = YA.rational_points()[0]
        v = vector(YA)
        SA[j] = [y[i].substitute(v) for i in range(N)]
    r = [K.random_element() for i in range(2)]
    SA = [r[0]*SA[0][j]+r[1]*SA[1][j] for j in range(N)]
    MAinv.change_ring(K)
    SA = vector(MAinv*Matrix(K,N,1,SA)) # Signature
    return D,SA

In [29]:
% time D,SA = Signing()

CPU times: user 742 ms, sys: 68.7 ms, total: 811 ms
Wall time: 2.19 s


In [30]:

def Verification():
    
    # Verification

    V1 = []
    for f in D:
        V1.append(f.substitute(SA))
    V2 =[]
    for f in KA:
        V2.append(f.substitute(SA))
    V = V1+V2
    t=0
    for i in V:
        if i==0:
            t = t+1
    if t == len(V):
        return ("Signature is valid "), V
    else:
        return ("Signature is invalid, try again "),V
    

In [31]:
% time Verification()

CPU times: user 150 ms, sys: 4.05 ms, total: 154 ms
Wall time: 151 ms


('Signature is valid ', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])